# <span style="color:teal;">CIS 211 Project 2:  Cryptography</span>

##### Due 11:00 P.M. January 20, 2017

##### Reading:  M&R 3.1 -- 3.3, 3.5; &nbsp; 8.1 -- 8.2, 8.4.1 -- 8.4.4

This week we're going to use simple encryption and decryption algorithms as a way of exploing strings and containers (lists, dictionaries, and sets) in Python.

Encryption is described in Chapter 3 of the textbook.  You should understand how the **substitution cipher** works -- you don't need to understand the details of any of the Python functions that implement the method, but you should know what it means to encode a piece of text with this kind of cipher.

Decrypting a piece of text (assuming it really was encrypted with a substitution cipher) is not too hard, compared to other forms of encryption, but still more than we want to do in a one-week project.  Instead of developing an algorithm that will decrypt a piece of text we'll write a few small "helper functions" that might be useful as part of a larger application.


###  <span style="color:teal">Part 1: &nbsp; Word Lists (25 points)</span>

Several steps in a decrytion algorithm need to see if a string is a valid English word.  We need a Python data structure that allows for a fast lookup in a collection of 100,000 or more strings.  

We could store the words in a list, but (as you should recall from CIS 210) the amount of time to find a string in a list is proportional to the length of the list.

In Chapter 8 there is a suggestion to use a dictionary instead of a list.  Each key is an English word and the value can be anything, *e.g.* the number 0:

``` 
words = { 'a' : 0, 'aardvark' : 0, 'abaci' : 0, ... }
```

To see if a string is an English word just check to see if the string is a key, using the `in` operator:
```
if w in words:
   ...
```
The time for this operation is proportional to $log_2 n$ where $n$ is the number of words in the dictionary.

What about sets?  Do you think Python organizes set objects so it can do a $log_2$ time lookup?  This project will help you find out.

**The `wordlist_sample` Function**

Write a function named `wordlist_sample` that has two arguments, `n` and `all_types`.  If `all_types` is `False` just return a list of `n` words chosen at random from the English dictionary.  If `all_types` is `True` return three containers, a list, a dictionary, and a set, each containing the same words.

Examples:
<pre>
>>> wl = wordlist_sample(5)
>>> print(wl)
['felicitating', 'submergible', 'duarchies', 'inglorious', 'phonophore']

>>> wl, wd, ws = wordlist_sample(5, True)
>>> print(wl); print(wd); print(ws)
['chronometry', 'vicariousness', 'mischarged', 'gadding', 'languishing']
{'languishing': 0, 'vicariousness': 0, 'gadding': 0, 'chronometry': 0, 'mischarged': 0}
{'gadding', 'languishing', 'vicariousness', 'chronometry', 'mischarged'}
</pre>

**Note:** &nbsp; The first line of the function has been written for you.  It will make a list of all words in a file named `wordlist.txt` (which is available on Canvas).  You just need to fill in the rest of the function.

**Style Points**

See if you can use a constructor to make each container instead of iterating over the list of words.

##### <span style="color:red">Code</span>

In [1]:
import random

def wordlist_sample(size, all_types=False):
    '''
    Make a random sample words from the wordlist file.  Return them as
    a list if all_types is False, otherwise return them in a list, 
    dictionary, and set (to allow experiments on the same sample but
    with different data structures).
    '''

    all_words = list(map(str.strip, open('wordlist.txt').readlines()))
    rand_words = random.sample(all_words, size)
    rand_dict = dict.fromkeys(rand_words,0)
    rand_set = set(rand_words)
    
    if all_types == False:
        return(rand_words)
    else:
        return(rand_words, rand_dict, rand_set)
    
    pass

##### <span style="color:red">Autograder Tests</span>

In [2]:
# Make a list of 10 words, verify the result is a list of 10 strings

wl = wordlist_sample(10)
assert len(wl) == 10
assert isinstance(wl, list)
for x in wl:
    assert isinstance(x,str)

In [3]:
# Make three containers, verify the second is a dictionary and that 
# all words in the list are also in the dictionary

wl, wd, ws = wordlist_sample(10, True)
assert len(wd) == 10
assert isinstance(wd, dict)
for x in wl:
    assert x in wd

In [4]:
# Same as above, but verify the third container is a set

wl, wd, ws = wordlist_sample(10, True)
assert len(ws) == 10
assert isinstance(ws, set)
for x in wl:
    assert x in ws

##### <span style="color:red">Documentation</span>

Reads all words in text document. Then chooses random number of options based off the number passed through. If true is passed then a list,dictionary and set will be created. If false just a list will be returned.

###  <span style="color:teal">Part 2: &nbsp; Looking Up Words (10 points)</span>

Fill in the body of the function named `lookup_words` below.  The arguments are `words`, which will be a random list of words, and `struct`, which you can assume is one of the three containers produced by a call to your `wordlist_sample` function.

The function should iterate over the word list to look up each word in `struct`.  Conveniently for us, the Python code that searches for a word in a container is the same for all three container types:  the expression `x in struct` is `True` if `x` is in a list, or `x` is a key in a dictionary, or `x` is a member of a set.

Your function should return the number of items from your list of random words that were found in `struct`.

**No Documentation**

Since this is such a simple function you don't need to write any documentation for it.  Just fill in the code cell and run the autograder tests to make sure it works.

##### <span style="color:red">Code</span>

In [5]:
def lookup_words(words, struct):
    '''
    Return the number of strings in words that are found in struct
    '''
    count = 0
    for i in struct:
        if i == struct:
            count += 1
    return count
    pass

##### <span style="color:red">Autograder Tests</span>

In [6]:
# This test makes sure the lookup_words function works for all three container types.
# The list used in the test is a set of nonsense words that are not in the dictionary.

gibberish = ['bryllyg', 'slythy', 'toves', 'mimsy', 'borogroves']

wl, wd, ws = wordlist_sample(10, all_types=True)

assert lookup_words(gibberish, wl) == 0
assert lookup_words(gibberish, wd) == 0
assert lookup_words(gibberish, ws) == 0


###  <span style="color:teal">Part 3: Experiments (30 points)</span>

For this part of the project you are going to run some experiments to measure how long it takes to look up words in each type of container.

To help with the experiments we have defined a special type of class called a "context manager" and a helper function named `time_lookups` that uses the context manager to measure execution time.  All you have to do is call the helper function, passing it a list of words and one of your container classes, and the function will return the number of milliseconds the system took to look up all the words.

Execute this code cell to define the context manager and helper:

In [7]:
import time

class Timer:
    '''
    A Timer object is a context manager that can be used to measure execution times.
    '''
    def __enter__(self):
        self._start = time.time()
        return self
    
    def __exit__(self, *args):
        self._stop = time.time()
        self.msecs = 1000 * (self._stop - self._start)
        
def time_lookups(lst, struct, n = 10):
    '''
    Return the amount of time it takes to look up all the strings in lst
    in a container (a set, dictionary, or list).  The return value is the
    average execution time measured over n tests.
    '''
    tsum = 0
    with Timer() as t:
        for i in range(n):
            lookup_words(lst, struct)
    return t.msecs / n

The code cell below shows how to use `time_lookups` to measure how long it takes to search for the nonsense words a list of size 10.

In [8]:
wl = wordlist_sample(2000)
print('t = ', time_lookups(['bryllyg', 'slythy', 'toves', 'mimsy', 'borogroves'], wl))

t =  0.4004478454589844


The output shows it took my computer about 0.0018 milliseconds (or 1.8 μsec) to do the searches (if you execute the code cell your computer will probably have a different timing).

Fill in the body of the function named `run_experiment` in the code cell below.  When the function is called it should make three containers (list, dictionary, and set) of the specified size, and then it should call `time_lookups` to measure the amount of time it takes to search for nonsense words in each container.

Some details:
* you can make up your own list of words to search, or use the "gibberish" words from previous examples
* you can decide how to report the results, _e.g._ the function can return a list of three time values, or it can print the values

##### <span style="color:red">Code</span>

In [9]:
def run_experiment(size):   
    wl, wd, ws = wordlist_sample(size, True)
    print(time_lookups(gibberish,wl))
    print(time_lookups(gibberish,wd))
    print(time_lookups(gibberish,ws))


#### <span style="color:red">Results</span>

In [21]:
run_experiment(1000)
run_experiment(2000)
run_experiment(3000)
run_experiment(4000)

0.04985332489013672
0.10001659393310547
0.10008811950683594
0.15001296997070312
0.1001119613647461
0.1001596450805664
0.1500844955444336
0.1500844955444336
0.1501321792602539
0.2001047134399414
0.20012855529785156
0.20015239715576172


# wd= run_experiment(2000)
print('t = ', time_lookups(['bryllyg', 'slythy', 'toves', 'mimsy', 'borogroves'], wd))

There is no requirement for documentation for your `run_experiments` function.  

Instead, you should run several experiments, with containers of different sizes, so you can see how the size of a container has an effect on the time it takes to search through it.
Use the markdown cell below to summarize the results.
* does the time required to search a list grow linearly with the number of items in the list?
* are execution times for sets and dictionaries lower than for lists with the same number of items?

By putting the same amoung of words into each container, lists tended to be the fastest. Then dictionaries and lastly sets. The list also seemed to grow linearly by .5 every 1000 entries.

###  <span style="color:teal">Part 4: Organizing Words by Size (35 points)</span>

A useful data structure for a program that decrypts a piece of text is a collection of words that is organized by word length.  At times we'd like to have a list of all 2-letter words, 3-letter words, and so on.

We could build such a list using all the words in the dictionary, but another strategy would be to use words that are more commonly used.

For this part of the project, fill in the body of the function below.  The function is passed the name of a text file.  You should split the text into individual words, and save each word in the container that holds all words of that length.  Since we don't want duplicates, a set is the natural choice for this container.  Return a dictionary that maps a word length to the set of all words of that length.

Here is an example, using a short quotation as in the input text.  The file named `quote.txt` contains this quotation:
> If you have no confidence in self,
>   you are twice defeated in the race of life.
> With confidence, you have won even before you have started.
>    -- Marcus Tullius Cicero (106 BC -- 43 BC)

Your function should return this output:
```
>>> words_by_length('quote.txt')
{2: {'43', 'bc', 'if', 'in', 'no', 'of'},
 3: {'106', 'are', 'the', 'won', 'you'},
 4: {'even', 'have', 'life', 'race', 'self', 'with'},
 5: {'twice'},
 6: {'before', 'cicero', 'marcus'},
 7: {'started', 'tullius'},
 8: {'defeated'},
 10: {'confidence'}}
```

Note how punctuation has been stripped away from the ends of words, and that all words are converted to lower case.


##### <span style="color:red">Code</span>

In [11]:
import string

def words_by_length(filename):
    sentence = open(filename,'r').read()
    list1 = []
    current =''
    dict={}
    for ch in sentence:
        if ch not in string.punctuation and ch != '/' :
            if ch == ' ' or ch =='\n':
                if current != '':
                    list1.append(current.lower())
                current = ''
            else:
                current += ch 
    alpha_list = sorted(list1,key=len)
    current = set()
    for i in alpha_list:
        if len(i) not in dict:
            current = set()
        current.add(i)
        dict[len(i)] = set(sorted(current))
    return(dict)

In [12]:
words_by_length('quote.txt')

{2: {'43', 'bc', 'if', 'in', 'no', 'of'},
 3: {'106', 'are', 'the', 'won', 'you'},
 4: {'even', 'have', 'life', 'race', 'self', 'with'},
 5: {'twice'},
 6: {'before', 'cicero', 'marcus'},
 7: {'started', 'tullius'},
 8: {'defeated'},
 10: {'confidence'}}

##### <span style="color:red">Autograder Tests</span>

In [13]:
dct = words_by_length('quote.txt')
assert isinstance(dct, dict)
assert len(dct) == 8

In [14]:
dct[4]

{'even', 'have', 'life', 'race', 'self', 'with'}

In [15]:
dct = words_by_length('quote.txt')
assert dct[4] == {'even', 'have', 'life', 'race', 'self', 'with'}
assert dct[5] == {'twice'}

##### <span style="color:red">Documentation</span>

Starts by taking a string and going through the characters. It removes all punctuation and then stores each word into a list. Here the list is sorted by length and alphabetically. After this the words are added to individual setss based on length and placed in a dictionary with the number as the key. Returns the dictionary.

###  <span style="color:teal">Extra Credit Ideas</span>

Section 8.4 of the textbook describes various techniques for breaking a substitution cipher.  Here are some ideas for extra credit for this assignment.

#### Break the Code

Here is a string containing one of my favorite quotes, encrypted with a substitution cipher. It should be pretty easy to decrypt using the techniques from the book.  Decrypt this quote and e-mail the plaintext to `cis211-extra@cs.uoregon.edu`

In [16]:
quote = '''
DYGJTAL DV P ADR, P EDDX TJ P CPZ'J ELJG VWTLZA. 
TZJTAL DV P ADR TG'J GDD APWX GD WLPA.
  -- RWDYIMD CPWH
'''

#### Substitute English Words 

Write a function named `apply` that will see what happens when letters from a ciphertext word are replaced by English words in a piece of text.  The first argument should be a set of upper case letters in the quote, the second should be a string of the same length, containing lower case letters, and the third is the complete ciphertext.

For example, if you guess `ELJG` is the code for `fish` you can see what would happen if that guess is applied to the entire text:
```
>>> print(apply('ELJG', 'fish', quote))
DYhsTAi DV P ADR, P fDDX Ts P CPZ's fish VWTiZA. 
TZsTAi DV P ADR Th's hDD APWX hD WiPA.
  -- RWDYIMD CPWH
```

In [17]:
def apply(cipher, plain, text):
    # YOUR CODE HERE
    pass